In [1]:
# Parameter determining if results will be visualized in cells
VISUALIZATION = False

# Dataset choice
DATASET_NAME = "dataset_large.txt"

# Initial steps

## 1.Libraries and environment initialization

In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install graphframes
!pip install kneed

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u412-ga-1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [3]:
import os
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, Row, types as T
from pyspark.sql.functions import col, rank, row_number, split, regexp_replace, collect_list, hash as spark_hash, concat_ws, concat, lit, when, abs, explode, desc, size
from pyspark.sql.types import ArrayType, StructType, StructField, StringType, LongType, DoubleType, IntegerType
from pyspark.sql.functions import udf
import numpy as np
from collections import Counter
from random import shuffle
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import MinHashLSH
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from graphframes import GraphFrame
import networkx as nx
import matplotlib.pyplot as plt
from itertools import chain
from pyspark.ml.feature import PCA as PCA_Spark, VectorAssembler, MinMaxScaler#, StandardScaler, RobustScaler
from pyspark.ml.clustering import BisectingKMeans
from kneed import KneeLocator

## 2.Create & Configure Session

In [4]:
def create_session():
    # create the session
    conf = SparkConf()
    conf.setAppName("DIS-lab-1")    # Sets name of the Spark Application
    conf.setMaster("local[16]")    # Master URL. In this case local[*] uses all the available cores in the machine
    conf.set("spark.driver.memory", "10G")   # Memory allocated to driver process
    conf.set("spark.driver.maxResultSize", "6G")    # Maximum size of results that can be returned to driver
    conf.set("spark.executor.instances", "4")
    conf.set("spark.executor.cores", "4")
    conf.set("spark.executor.memory", "4G")    # Memory allocated to each executor
    conf.set("spark.network.timeout", "600s")  # Increase network timeout
    conf.set("spark.executor.heartbeatInterval", "60s")  # Increase heartbeat interval
    conf.set("spark.rpc.message.maxSize", "512")  # Increase max message size
    conf.set("spark.driver.maxResultSize", "4G")  # Increase driver max result size
    conf.set("spark.sql.broadcastTimeout", "600")  # Increase broadcast timeout
    conf.set("spark.sql.shuffle.partitions", "200")  # Increase the number of shuffle partitions
    conf.set("spark.yarn.executor.memoryOverhead", "2048")  # Increase memory overhead
    conf.set("spark.memory.offHeap.enabled","true")
    conf.set("spark.memory.offHeap.size","10g")
    conf.set("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.0-s_2.12")  # Add GraphFrames to the spark configuration
    conf.set("spark.hadoop.fs.file.impl", "org.apache.hadoop.fs.LocalFileSystem")  # Ensure local file system is used for checkpointing

    sc = pyspark.SparkContext(conf=conf)    # Initializes the Spark context with this specific configuration
    spark = SparkSession.builder.config(conf=sc.getConf()).getOrCreate()    # Creates Spark session

    # Set checkpoint directory
    sc.setCheckpointDir("checkpoints")

    return sc, spark

try:
    if 'sc' in globals() and sc is not None:
        sc.stop()
        print("--Stopped existing SparkContext")
    if 'spark' in globals() and isinstance(spark, SparkSession):
        spark.stop()
        print("--Stopped existing SparkSession")
except Exception as e:
    print(f"Error stopping existing Spark session or context: {e}")

# Create a new Spark session
sc, spark = create_session()
print("Spark session created successfully!")
spark

Spark session created successfully!


## 3.Load and Split Data

### Load the data from the text file into Dataframe

In [5]:
logs_txt = spark.sparkContext.textFile(DATASET_NAME)
logs = logs_txt.map(lambda x: (x, )).toDF(["Logs"])

### Split the data into 5 separate columns

In [6]:
logs_splitted = logs \
    .withColumn("from_server", regexp_replace(split(col("Logs"), ", ").getItem(0), "[<>]", "")) \
    .withColumn("to_server", split(col("Logs"), ", ").getItem(1)) \
    .withColumn("time", split(col("Logs"), ", ").getItem(2)) \
    .withColumn("action", split(col("Logs"), ", ").getItem(3)) \
    .withColumn("process_id", regexp_replace(split(col("Logs"), ", ").getItem(4), "[<>]", "")) \
    .drop("Logs")

# Cast the "time" and "process_id" columns to integers
logs_casted = logs_splitted \
    .withColumn("time", col("time").cast("integer")) \
    .withColumn("process_id", col("process_id").cast("integer"))

if VISUALIZATION:
  logs_splitted.show(1, truncate=False)

## 4.Prepare data for processing

### Group by process_id

In [7]:
logs_grouped = logs_casted.groupBy("process_id").agg(
    collect_list("from_server").alias("from_servers"),
    collect_list("to_server").alias("to_servers"),
    collect_list("time").alias("times"),
    collect_list("action").alias("actions")
)
logs_grouped_not_sorted = logs_grouped

if VISUALIZATION:
  logs_grouped.show(6, truncate=False)

### Sort all the column items based on 'from_server' so the sub-processes are in the right order

In [8]:
def sort_lists_time(times, from_servers, to_servers, actions):
    combined = list(zip(times, from_servers, to_servers, actions))
    sorted_combined = sorted(combined, key=lambda x: x[0])

    # Remove the second half of each list since it contains duplicte names
    half_length = len(sorted_combined) // 2
    truncated_combined = sorted_combined[:half_length]

    times_sorted, from_servers_sorted, to_servers_sorted, actions_sorted = zip(*truncated_combined)
    return list(times_sorted), list(from_servers_sorted), list(to_servers_sorted), list(actions_sorted)

def sort_lists_from_server(times, from_servers, to_servers, actions):
    # First sort by time
    combined = list(zip(times, from_servers, to_servers, actions))
    sorted_combined = sorted(combined, key=lambda x: x[0])  # Sort by times (x[0])

    # Remove the second half of each list since it contains duplicte names
    half_length = len(sorted_combined) // 2
    truncated_combined = sorted_combined[:half_length]

    times_sorted, from_servers_sorted, to_servers_sorted, actions_sorted = zip(*truncated_combined)

    combined = list(zip(times_sorted, from_servers_sorted, to_servers_sorted, actions_sorted))
    sorted_combined = sorted(combined, key=lambda x: x[1])
    times_sorted, from_servers_sorted, to_servers_sorted, actions_sorted = zip(*sorted_combined)
    return list(times_sorted), list(from_servers_sorted), list(to_servers_sorted), list(actions_sorted)

# Define the schema for the sorted columns
sorted_lists_schema = StructType([
    StructField("times", ArrayType(LongType()), nullable=True),
    StructField("from_servers", ArrayType(StringType()), nullable=True),
    StructField("to_servers", ArrayType(StringType()), nullable=True),
    StructField("actions", ArrayType(StringType()), nullable=True)
])

# Register the function as a UDF
sort_lists_udf = udf(sort_lists_from_server, sorted_lists_schema)

# Apply the UDF to sort the lists based on the "times" column
logs_grouped = logs_grouped.withColumn("sorted_lists", sort_lists_udf("times", "from_servers", "to_servers", "actions"))

# Split the sorted lists into separate columns
logs_grouped = logs_grouped.withColumn("times", col("sorted_lists.times")) \
                           .withColumn("from_servers", col("sorted_lists.from_servers")) \
                           .withColumn("to_servers", col("sorted_lists.to_servers")) \
                           .withColumn("actions", col("sorted_lists.actions")) \
                           .drop("sorted_lists")

### Create 'process_string' column with a string of all the 'from_servers'

In [9]:
'''
Using the withColumn method to create a new column "process_string".
The concat_ws function concatenates multiple column values into a single string, separated by a comma.
col("from_servers"): Selects the column "from_servers".
col("to_servers"): Selects the column "to_servers".
col("times"): Selects the column "times".
col("actions"): Selects the column "actions".
The concatenated string is stored in the new column "process_string".
'''

logs_grouped = logs_grouped.withColumn(
    "process_string",
    concat_ws(",", col("from_servers"))
)

# Part 1: "Cleaning": LSH and Connected Components

## Step 1. Create shingles

### 1.1. Pre-processing before shingles

#### Find common sub-string

In [10]:
def get_all_substrings(string, min_length):
    length = len(string)
    return [string[i:j] for i in range(length) for j in range(i + min_length, length + 1)]


def find_common_substring(server_names, min_length=3, min_occurrences_ratio=0.6):
    substr_counter = Counter()
    total_names = len(server_names)
    min_occurrences = int(total_names * min_occurrences_ratio)

    for name in server_names:
        substrings = get_all_substrings(name, min_length)
        substr_counter.update(substrings)

    # Filter substrings that occur at least min_occurrences times
    common_substrings = [substr for substr, count in substr_counter.items() if count >= min_occurrences]

    if not common_substrings:
        return ""

    # Return the longest common substring
    return max(common_substrings, key=len)


def remove_common_substring(strings, common_substring):
    return [string.replace(common_substring, "") for string in strings]

In [11]:
# Sample 5 rows from the data
sample_rows = logs_grouped.select("from_servers").take(3)

# Extract the "from_servers" lists from the sample rows
sample_from_servers = [row["from_servers"] for row in sample_rows]

# Flatten the list of lists to a single list of server names
flattened_server_names = [server for sublist in sample_from_servers for server in sublist]

# Get unique server names
unique_server_names = list(set(flattened_server_names))

# Use the function on the unique server names
common_substring = find_common_substring(unique_server_names, min_length=3, min_occurrences_ratio=0.70)

if VISUALIZATION:
  print("Most Common Substring:", common_substring)

#### Remove common sub-string + special characters + spaces

In [12]:
# Create a new DataFrame with the common substring removed from the process_string
logs_grouped_cleaned = logs_grouped.withColumn(
    "process_string_clean_common",
    regexp_replace(col("process_string"), common_substring, ""),
)

logs_grouped_cleaned = logs_grouped_cleaned.withColumn(
    "process_string_cleaned",
    regexp_replace(col("process_string_clean_common"), "[^a-zA-Z0-9\s]", "")
)

# Show the cleaned DataFrame
if VISUALIZATION:
  logs_grouped_cleaned.select("process_string_cleaned").limit(1).show(truncate=False)

### 1.2. Define the shingles of each function

In [13]:
# Define the shingling function for process strings
def shingle_process_string(process_string, k):
    shingle_set = set()
    for i in range(len(process_string) - k + 1):
        shingle = process_string[i:i + k]
        shingle_set.add(shingle)
    return list(shingle_set)

# Define the shingle length based on the average length of the servers of the first 5 rows
first_rows = logs_grouped_cleaned.take(5)
average_length = 0

# Calculate the average length of server names
total_length = 0
total_servers = 0

# For each row in the sample rows sum the length of the names and the number of servers
for row in first_rows:
    servers = row["process_string_clean_common"]
    server_names = servers.split(',')
    total_length += sum([len(server) for server in server_names])
    total_servers += len(server_names)

# Find the average server name length
average_server_name_length = total_length / total_servers

# Devine ths shingle length as the 30% of the average server name (or 2 if the names are too short)
shingle_length_percentage = 0.3
shingle_length = max(2, int(np.ceil(average_server_name_length *shingle_length_percentage)))

if VISUALIZATION:
  print(f'Average length of server names: {average_server_name_length}')
  print(f'Determined shingle length: {shingle_length}')

# Register the UDF for shingling
shingle_udf = udf(lambda process_string: shingle_process_string(process_string, shingle_length), ArrayType(StringType()))

# Apply the UDF to create shingles from process strings
logs_grouped_cleaned_with_shingles = logs_grouped_cleaned.withColumn("shingles", shingle_udf(col("process_string_cleaned")))

# Show the resulting DataFrame with shingles
if VISUALIZATION:
  logs_grouped_cleaned_with_shingles.select("process_id", "process_string_cleaned", "shingles").limit(1).show(truncate=False)

### 1.3. Create Vocabulary and Sparse Vectors

#### Keep all unique shingles

In [14]:
# Collect all shingles from the DataFrame into a single list
all_unique_shingles = logs_grouped_cleaned_with_shingles.select("shingles").rdd.flatMap(lambda row: row.shingles).distinct().collect()

# Create a vocabulary dictionary where each shingle is assigned a unique index
vocab = {shingle: idx for idx, shingle in enumerate(all_unique_shingles)}
# Print the vocabulary to verify
if VISUALIZATION:
  print(f'Vocabulary: {str(list(vocab)[:10])[:-1]} . . . ]')
  print(f'Length: {len(vocab)}')

#### Create the sparse vector of shingle occurence for each process

In [15]:
# Define a function to create sparse vectors based on the vocabulary
def create_sparse_vector_for_lsh(shingles, vocab):
    indices = []
    for shingle in shingles:
        if shingle in vocab:
            idx = vocab[shingle]
            indices.append(idx)

    indices = sorted(indices)  # Sort indices
    values = [1.0] * len(indices)
    return Vectors.sparse(len(vocab), indices, values)

# Register the UDF to create sparse vectors for LSH
sparse_vector_for_lsh_udf = udf(lambda shingles: create_sparse_vector_for_lsh(shingles, vocab), VectorUDT())

# Apply the UDF to create sparse vectors directly suitable for MinHashLSH
logs_sparse_vectors = logs_grouped_cleaned_with_shingles.withColumn("features", sparse_vector_for_lsh_udf(col("shingles")))

# Show the resulting DataFrame with sparse vectors
if VISUALIZATION:
  logs_sparse_vectors.select("process_id", "features").limit(1).show(truncate=False)

## Step 2. MinHash and LSH with pyspark's MinHashLSH

### 2.1. Create MinHashLSH model and fit it with the 'features' column of the 'logs_sparse_vectors' dataframe

In [16]:
# Initialize MinHashLSH
'''
:inputCol: The column of our dataframe that is going to be used for hashing
:outputCol: The new solumn that will be created that includes the hashes
:numHashTables: Number of MinHash functions
:seed: Seed for reproducibility

'''
mh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=30, seed=12345)

# Fit the model
model = mh.fit(logs_sparse_vectors)

# Transform the data to include the hashes
transformed_df = model.transform(logs_sparse_vectors)

# Show the resulting DataFrame with hashes
if VISUALIZATION:
  transformed_df.select("process_id", "hashes").limit(1).show(truncate=False)

### 2.2. Find pairs with Jaccard distance < threshold

In [17]:
# Find similar pairs with Jaccard similarity above a threshold
max_distance_threshold = 0.425
similar_pairs_df = model.approxSimilarityJoin(transformed_df, transformed_df, max_distance_threshold, distCol="JaccardDistance") \
    .select(col("datasetA.process_id").alias("process_id_1"),
            col("datasetB.process_id").alias("process_id_2"),
            col("JaccardDistance"))

# Filter out rows where process_id_1 is equal to process_id_2
filtered_pairs_df = similar_pairs_df.filter(col("process_id_1") != col("process_id_2"))

if VISUALIZATION:
  filtered_pairs_df.select("JaccardDistance").describe().show()
  filtered_pairs_df.show()

## Step 3. Group similar processes with connected components in graph and update dataset

### 3.1. Construct graph using similarities between nodes as edges and find connected components

In [18]:
# Create GraphFrame
edges = filtered_pairs_df.select(col("process_id_1").alias("src"),
                                col("process_id_2").alias("dst"),
                                col("JaccardDistance").alias("weight"))
vertices = edges.selectExpr("src as id").union(edges.selectExpr("dst as id")).distinct()
graph = GraphFrame(vertices, edges)

# Find Connected Components
connected_components = graph.connectedComponents()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [19]:
# Construct NetworkX graph
if VISUALIZATION:
  nx_graph = nx.Graph()
  for row in graph.edges.collect():
      nx_graph.add_edge(row['src'], row['dst'])

  # Visualize using matplotlib
  pos = nx.spring_layout(nx_graph)
  nx.draw(nx_graph, pos, with_labels=True, node_color='skyblue', node_size=500,
          edge_color='#909090', width=0.5, font_size=8, font_color='#7c7c7c',
          font_weight='bold')
  plt.show()

  print()

  pos = nx.fruchterman_reingold_layout(nx_graph)
  nx.draw(nx_graph, pos, with_labels=True, node_color='skyblue', node_size=500,
          edge_color='#909090', width=0.5, font_size=8, font_color='#7c7c7c',
          font_weight='bold')
  plt.show()

### 3.2. Keep only one out of each group of similar processes and create the final filtered dateset

In [20]:
# Define the window specification for collecting members
window_spec_collect = Window.partitionBy("component")

# Define the window specification for ordering by id
window_spec_representative = Window.partitionBy("component").orderBy("id")

# Use row_number to get the representative
representatives_with_row_num = connected_components \
    .withColumn("row_num", row_number().over(window_spec_representative))

# Collect all ids of the connected components into a list, excluding the representative
connected_components_with_list = representatives_with_row_num \
    .withColumn("members", collect_list(
        when(col("row_num") != 1, col("id"))
    ).over(window_spec_collect))

# Filter for the representative and select the required columns
groups_with_members = connected_components_with_list \
    .where(col("row_num") == 1) \
    .select("id", "members")

if VISUALIZATION:
  groups_with_members.show()

In [21]:
# Collect all the member lists into a single list
member_lists = groups_with_members.select("members").rdd.flatMap(lambda x: x).collect()

# Flatten the list of lists into a single list
ids_to_drop = list(chain.from_iterable(member_lists))

if VISUALIZATION:
  print(ids_to_drop)

In [22]:
# Delete grouped processes
filtered_df = logs_grouped_not_sorted.filter(~col("process_id").isin(ids_to_drop))

filtered_df_strings = filtered_df.select(
    col("process_id"),
    concat(lit("["), concat_ws(",", col("from_servers")), lit("]")).alias("from_servers"),
    concat(lit("["), concat_ws(",", col("to_servers")), lit("]")).alias("to_servers"),
    concat(lit("["), concat_ws(",", col("times")), lit("]")).alias("times"),
    concat(lit("["), concat_ws(",", col("actions")), lit("]")).alias("actions")
)

if VISUALIZATION:
  filtered_df_strings.show(1, truncate=False)

## Step 4. Export part1Output.txt & part1Observations.txt

In [23]:
with open("part1Observations.txt", "w") as f:
    grouped = []
    for row in groups_with_members.collect():
        representative_id = row["id"]
        member_ids = row["members"]
        group_ids = [representative_id] + member_ids
        grouped = grouped + group_ids

        f.write(f"Group: {{{', '.join(map(str, group_ids))}}}\n")
        for process_id in group_ids:
            f.write(f"{process_id}:\n")
            # Filter logs directly for the current process_id
            process_logs = logs_grouped_not_sorted.filter(col("process_id") == process_id).collect()
            for log in process_logs:
                from_servers = log['from_servers']
                to_servers = log['to_servers']
                times = log['times']
                actions = log['actions']
                process_id_val = log['process_id']

                # Iterate through the lists and create log entries
                for from_server, to_server, time, action in zip(from_servers, to_servers, times, actions):
                    log_str = f"<{from_server}, {to_server}, {time}, {action}, {process_id_val}>"
                    f.write(f"\t\t\t{log_str}\n")

        f.write("\n")

    ungrouped_df = logs_grouped_not_sorted.filter(~col("process_id").isin(grouped))
    for row in ungrouped_df.collect():
        process_id_val = row['process_id']
        process_logs = logs_grouped_not_sorted.filter(col("process_id") == process_id_val).collect()
        for log in process_logs:
          from_servers = log['from_servers']
          to_servers = log['to_servers']
          times = log['times']
          actions = log['actions']

          f.write(f"Group: {{{process_id_val}}}\n")
          f.write(f"{process_id_val}:\n")
          # Iterate through the lists and create log entries
          for from_server, to_server, time, action in zip(from_servers, to_servers, times, actions):
              log_str = f"<{from_server}, {to_server}, {time}, {action}, {process_id_val}>"
              f.write(f"\t\t\t{log_str}\n")

In [24]:
with open("part1Output.txt", "w") as f:
    for row in filtered_df.collect():
        process_id_val = row['process_id']
        process_logs = logs_grouped_not_sorted.filter(col("process_id") == process_id_val).collect()
        for log in process_logs:
          from_servers = log['from_servers']
          to_servers = log['to_servers']
          times = log['times']
          actions = log['actions']

          # Iterate through the lists and create log entries
          for from_server, to_server, time, action in zip(from_servers, to_servers, times, actions):
              log_str = f"<{from_server}, {to_server}, {time}, {action}, {process_id_val}>"
              f.write(f"{log_str}\n")

In [25]:
print(f'The original dataset had: {logs_grouped.count()} processes and we kept: {filtered_df.count()}')
print(f'Groups created: {groups_with_members.count()}')

The original dataset had: 1000 processes and we kept: 221
Groups created: 20


# Part 2: "Categorization"

## Import & format data from Part 1

In [26]:
logs_txt = spark.sparkContext.textFile("part1Output.txt")
logs = logs_txt.map(lambda x: (x, )).toDF(["Logs"])

logs_splitted = logs \
    .withColumn("from_server", regexp_replace(split(col("Logs"), ", ").getItem(0), "[<>]", "")) \
    .withColumn("to_server", split(col("Logs"), ", ").getItem(1)) \
    .withColumn("time", split(col("Logs"), ", ").getItem(2)) \
    .withColumn("action", split(col("Logs"), ", ").getItem(3)) \
    .withColumn("process_id", regexp_replace(split(col("Logs"), ", ").getItem(4), "[<>]", "")) \
    .drop("Logs")

# Cast the "time" and "process_id" columns to integers
logs_casted = logs_splitted \
    .withColumn("time", col("time").cast("integer")) \
    .withColumn("process_id", col("process_id").cast("integer"))

# Group
logs_grouped = logs_casted.groupBy("process_id").agg(
    collect_list("from_server").alias("from_servers"),
    collect_list("to_server").alias("to_servers"),
    collect_list("time").alias("times"),
    collect_list("action").alias("actions")
)
logs_grouped_not_sorted = logs_grouped

# Define the schema for the sorted columns
sorted_lists_schema = StructType([
    StructField("times", ArrayType(LongType()), nullable=True),
    StructField("from_servers", ArrayType(StringType()), nullable=True),
    StructField("to_servers", ArrayType(StringType()), nullable=True),
    StructField("actions", ArrayType(StringType()), nullable=True)
])

# Register the function as a UDF
sort_lists_udf = udf(sort_lists_from_server, sorted_lists_schema)

# Apply the UDF to sort the lists based on the "times" column
logs_grouped = logs_grouped.withColumn("sorted_lists", sort_lists_udf("times", "from_servers", "to_servers", "actions"))

# Split the sorted lists into separate columns
logs_grouped = logs_grouped.withColumn("times", col("sorted_lists.times")) \
                           .withColumn("from_servers", col("sorted_lists.from_servers")) \
                           .withColumn("to_servers", col("sorted_lists.to_servers")) \
                           .withColumn("actions", col("sorted_lists.actions")) \
                           .drop("sorted_lists")

logs = logs_grouped.dropDuplicates(['process_id'])
logs = logs_grouped.drop(*['to_servers', 'actions'])

if VISUALIZATION:
  logs.limit(5).show(truncate=False)

## Step 1. Create features for clustering

### 1.1 Create total and average time taken features

In [27]:
# Get total time taken
logs = logs.withColumn('sum_times', F.expr("aggregate(times, CAST(0 AS bigint), (acc, x) -> acc + x)"))

# Get average time taken for each server
logs = logs.withColumn('avg_times', F.expr("sum_times / size(times)"))

logs = logs.drop("times")

if VISUALIZATION:
  logs.limit(5).show(truncate=False)

### 1.2 Create server hops and base server names as one hot encoding features

In [28]:
# Get server hops
logs = logs.withColumn('server_hops', F.size(F.col('from_servers')))

"""
# Get unique server names
unique_servers = logs.select(F.explode('from_servers').alias('server')).distinct()

# Pivot to create one-hot encoded columns for servers
logs = logs.join(unique_servers, F.array_contains(logs['from_servers'], unique_servers['server']), 'left_outer') \
           .groupBy(logs.columns) \
           .pivot('server') \
           .agg(F.lit(1).alias('dummy')) \
           .na.fill(0)
"""

# Remove numbers to get base server names, filter out base servers that are also in from_servers
logs = logs.withColumn('base_servers', F.expr("transform(from_servers, x -> regexp_replace(x, '[0-9]+', ''))"))
logs = logs.withColumn('base_servers', F.array_except('base_servers', 'from_servers'))
logs = logs.drop('from_servers')

# Get unique base server names
unique_base_servers = logs.select(F.explode('base_servers').alias('base_server')).distinct()

# Pivot to create one-hot encoded columns for base servers
logs = logs.join(unique_base_servers, F.array_contains(logs['base_servers'], unique_base_servers['base_server']), 'left_outer') \
           .groupBy(logs.columns) \
           .pivot('base_server') \
           .agg(F.lit(1).alias('dummy')) \
           .na.fill(0)

logs = logs.drop('base_servers')

if VISUALIZATION:
  logs.limit(5).show(truncate=False)

### 1.3. Scale features that aren't one hot encoded and vectorize

In [29]:
# Scale features
assembler = VectorAssembler(inputCols=['sum_times', 'avg_times', 'server_hops'], outputCol='features_to_scale')
feature_logs = assembler.transform(logs)
scaler = MinMaxScaler(inputCol='features_to_scale', outputCol='scaled_features')
scaler_model = scaler.fit(feature_logs)
feature_logs = scaler_model.transform(feature_logs)
"""
scaler = StandardScaler(inputCol='features_to_scale', outputCol='scaled_features', withMean=True, withStd=True)
scaler_model = scaler.fit(feature_logs)
feature_logs = scaler_model.transform(feature_logs)
scaler = RobustScaler(inputCol='features_to_scale', outputCol='scaled_features')
scaler_model = scaler.fit(feature_logs)
feature_logs = scaler_model.transform(feature_logs)
"""
feature_logs = feature_logs.drop(*['sum_times', 'avg_times', 'server_hops', 'features_to_scale'])

# Assemble features into a single vector column
assembler = VectorAssembler(inputCols=[col for col in feature_logs.columns if col != 'process_id'], outputCol='features')
feature_logs = assembler.transform(feature_logs)

if VISUALIZATION:
  feature_logs.limit(5).show(truncate=False)

### 1.4 Reduce features to 2D using PCA

In [30]:
  pca = PCA_Spark(k=2, inputCol='features', outputCol='pca_features')
  pca_model = pca.fit(feature_logs)
  feature_logs = pca_model.transform(feature_logs)

  if VISUALIZATION:
    feature_logs.limit(5).show(truncate=False)

## Step 2. Cluster using Bisecting K-means

### 2.1 Cluster features several times with batches to find optimal k with Elbow method

In [31]:
# Define the range of k values to test
batch_size = 10
min_k = 2
max_k = logs.count()
# Ensure batch size of k for correct elbow method results
k_values = list(range(min_k, max(max_k + 1, min_k + batch_size)))
print(f'Search for k: {k_values}')

# Function to run the clustering in batches
def run_batches(k_values, batch_size, features='features'):
    wssse_values = []
    previous_optimal_k = None
    while k_values:
        batch_k = k_values[:batch_size]
        k_values = k_values[batch_size:]

        for k in batch_k:
            kmeans = BisectingKMeans(k=k, seed=50, featuresCol=features)
            model = kmeans.fit(feature_logs)
            wssse = model.summary.trainingCost
            wssse_values.append((k, wssse))

        # Extract K and WSSSE values
        k_vals, wssse_vals = zip(*wssse_values)

        # Use the Kneedle algorithm to find the elbow point
        kneedle = KneeLocator(k_vals, wssse_vals, curve='convex', direction='decreasing')
        optimal_k = kneedle.knee

        if optimal_k == previous_optimal_k:
            break
        previous_optimal_k = optimal_k

    return optimal_k, k_vals, wssse_vals

# Run the clustering in batches
optimal_k_nonpca, k_vals_nonpca, wssse_vals_nonpca = run_batches(k_values, batch_size)
print(f'The optimal number of clusters with normal features is: {optimal_k_nonpca}')

if VISUALIZATION:
  # Plot the WSSSE values against the k values
  plt.figure(figsize=(8, 6))
  plt.plot(k_vals_nonpca, wssse_vals_nonpca, 'bx-')
  plt.xlabel('Number of clusters (k)')
  plt.xticks(k_vals_nonpca)
  plt.ylabel('WSSSE')
  plt.title('Elbow Method For Optimal k')
  plt.show()

Search for k: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 2

### 2.2 Cluster PCA features several times with batches to find optimal k with Elbow method

In [32]:
# Ensure batch size of k for correct elbow method results
k_values = list(range(min_k, max(max_k + 1, min_k + batch_size)))
print(f'Search for k: {k_values}')

# Run the clustering in batches
optimal_k_pca, k_vals_pca, wssse_vals_pca = run_batches(k_values, batch_size, features="pca_features")
print(f'The optimal number of clusters with 2D PCA features is: {optimal_k_pca}')

if VISUALIZATION:
  # Plot the WSSSE values against the k values
  plt.figure(figsize=(8, 6))
  plt.plot(k_vals_pca, wssse_vals_pca, 'bx-')
  plt.xlabel('Number of clusters (k)')
  plt.xticks(k_vals_pca)
  plt.ylabel('WSSSE')
  plt.title('Elbow Method For Optimal k with 2D PCA features')
  plt.show()

Search for k: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 2

### 2.3 Choose optimal k and best features

In [33]:
# Smaller k and less error preferred
k_idx_nonpca = k_vals_nonpca.index(optimal_k_nonpca)
k_idx_pca = k_vals_pca.index(optimal_k_pca)
if optimal_k_nonpca <= optimal_k_pca and wssse_vals_nonpca[k_idx_nonpca] <= wssse_vals_pca[k_idx_pca]:
  optimal_k = optimal_k_nonpca
  features = 'features'
  print(f'The optimal number of clusters is {optimal_k_nonpca} and uses normal features')
else:
  optimal_k = optimal_k_pca
  features = 'pca_features'
  print(f'The optimal number of clusters is {optimal_k_pca} and uses 2D PCA features')

The optimal number of clusters is 7 and uses 2D PCA features


### 2.4 Cluster with optimal k and best features

In [34]:
# Train the final model using the optimal k
kmeans = BisectingKMeans(k=optimal_k, seed=123, featuresCol=features)
model = kmeans.fit(feature_logs)

# Make predictions
predictions = model.transform(feature_logs)

# Show cluster centers
"""
print("Cluster Centers:")
centers = model.clusterCenters()
for idx, center in enumerate(centers):
    print(f"Cluster {idx}: {center}")
"""

# Group predictions
grouped_predictions = predictions.groupBy('prediction').agg(collect_list('process_id').alias('process_ids')).orderBy('prediction', ascending=True)

# Show predictions
if VISUALIZATION:
  predictions.select('process_id', 'prediction').show()
  grouped_predictions.show(truncate=False)

In [35]:
if VISUALIZATION:
  # Extract the PCA result and cluster assignments
  pca_features = predictions.select('pca_features').rdd.map(lambda row: row[0]).collect()
  pca_features = np.array(pca_features)
  cluster_assignments = predictions.select('prediction').rdd.map(lambda row: row[0]).collect()

  # Plot the PCA result in 2D
  plt.figure(figsize=(10, 7))
  clusters = np.unique(cluster_assignments)
  for cluster in clusters:
      cluster_indices = np.where(np.array(cluster_assignments) == cluster)
      plt.scatter(pca_features[cluster_indices, 0], pca_features[cluster_indices, 1], label=f'Cluster {cluster}')
  plt.xlabel('PCA Component 1')
  plt.ylabel('PCA Component 2')
  plt.title('PCA of Clustered Data in 2D')
  plt.legend()
  plt.show()

## Step 3. Export part2Observations.txt

In [36]:
with open("part2Observations.txt", "w") as f:
    grouped = []
    for row in grouped_predictions.collect():
        representative_id = row["prediction"]
        group_ids = sorted(row["process_ids"])

        f.write(f"Group: {{{', '.join(map(str, group_ids))}}}\n")
        for process_id in group_ids:
            f.write(f"{process_id}:\n")
            # Filter logs directly for the current process_id
            process_logs = logs_grouped_not_sorted.filter(col("process_id") == process_id).collect()
            for log in process_logs:
                from_servers = log['from_servers']
                to_servers = log['to_servers']
                times = log['times']
                actions = log['actions']
                process_id_val = log['process_id']

                # Iterate through the lists and create log entries
                for from_server, to_server, time, action in zip(from_servers, to_servers, times, actions):
                    log_str = f"<{from_server}, {to_server}, {time}, {action}, {process_id_val}>"
                    f.write(f"\t\t\t{log_str}\n")

        f.write("\n")